In [241]:
import cv2

In [242]:
def find_large_label(img, img_bin):
    test_img = img.copy()
    _, labels, stats, centroids = cv2.connectedComponentsWithStats(img_bin)
    stats = sorted(stats, key = lambda x: x[4],reverse=True)
    # for i, rec in enumerate(stats[0:7]):
    #     x,y,w,h, area = rec
    #     cv2.rectangle(test_img, (x,y,w,h), (0,0,255))

    # cv2.imshow('items', img_resize(test_img, 600))
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # print(stats[0])
    return stats[1]

In [243]:
def get_threshold(img,gray,bin_inverse = True):
    if bin_inverse:
        _, img_bin = cv2.threshold(gray, -1,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    else:
        _, img_bin = cv2.threshold(gray, -1,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    return img_bin


In [244]:
def preproces(img, bin_inverse = True, only_bin = False):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_bin = get_threshold(img,gray,bin_inverse = True)

    large_stat = find_large_label(img, img_bin)
    try:
        item_img = img[large_stat[1]:large_stat[1]+large_stat[3],large_stat[0]:large_stat[0]+large_stat[2], :].copy()
        item_gray = gray[large_stat[1]:large_stat[1]+large_stat[3],large_stat[0]:large_stat[0]+large_stat[2]].copy()
    except:
        item_img, item_gray= img,gray

    item_bin = get_threshold(item_img,item_gray,bin_inverse = False)
    return  item_img, item_gray, item_bin

In [388]:
img_ok = cv2.imread('../team/images/true_ok/GSY827AN7A1356_AAO11960K_PKT10_CM1EQSUA0012_20220711210457_DirectLight_OK.jpg')
img_ng = cv2.imread('../team/images/true_ng/GSY827AN7B0519_AAO12705K_PKT08_CM1EQSUA0011_20220711213213_DirectLight_NG.jpg')
img_ng2 = cv2.imread('../team/images/true_ng/GSY827AN7A1641_AAO19043K_PKT13_CM1EQSUA0012_20220711220653_DirectLight_NG.jpg')
img_over = cv2.imread('../team/images/overkill/GSY827AN7B0355_AAO04537K_PKT01_CM1EQSUA0011_20220711234650_DirectLight_NG.jpg')


In [246]:
img_ng = cv2.imread('../team/images/true_ng/GSY827AN7D2341_AAO08637K_PKT11_CM1EQSUA0012_20220714014230_DirectLight_NG.jpg')

In [247]:
import numpy as np
def img_mask(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(img_hsv,(0,0,0),(0,0,100))
    imask = mask>0
    col = np.zeros_like(img_hsv, np.uint8)
    col[imask] = img_hsv[imask]
    col = cv2.cvtColor(col,cv2.COLOR_HSV2BGR)
    return col

In [338]:
import math
# 두 점의 각도 계산
def get_angle(pt1, pt2):
    xd = abs(pt2[0] - pt1[0])
    yd = abs(pt2[1] - pt1[1])
    try:
        radian = math.atan(yd/xd)
    except:
        radian = 0.5*math.pi
    return radian


In [353]:
get_angle([0,0], [1,-(3**0.5)])*180/math.pi

59.99999999999999

In [302]:
# 각도를 계산할 두 꼭짓점 찾기
def get_points(cnt):
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)   # 중심점과 각도를 4개의 꼭지점 좌표로 변환
    box = np.int0(box)          # 정수로 변환
    # pt1_ind = cnt[:,:,1].argmin()
    # pt2_ind = cnt[:,:,0].argmin()

    return box

In [276]:
math.sin(1/6*math.pi)

0.49999999999999994

In [385]:
def cnt_test(cnt):
    volum_ratio_bound = 0.005
    """_summary_

    Args:
        cnt (_type_): _description_

    Returns:
        _type_: _description_
    """
    w_max, w_min, h_max, h_min = (
        cnt[:, :, 0].max(),
        cnt[:, :, 0].min(),
        cnt[:, :, 1].max(),
        cnt[:, :, 1].min(),
    )
    # print(w_max - w_min, h_max - h_min)
    area = cv2.contourArea(cnt)
    arcLen = cv2.arcLength(cnt, closed=True)
    box = get_points(cnt)
    angle = get_angle(box[1], box[2])
    # print(max_area, area,round(area/max_area,3), arcLen)
    sin, cos = math.sin(angle), math.cos(angle)
    cal_max_box = ((arcLen**2)/4 - 2*area)*(sin*cos)+area
    max_box = (w_max-w_min)*(h_max-h_min)
    print(round(cal_max_box/max_box,3), angle*180/math.pi)
    # print(box[1:3], angle*180/math.pi)
    if  cal_max_box/max_box < 1-volum_ratio_bound or cal_max_box/max_box > 1+volum_ratio_bound :
        pred = 'NG'
    else:
        pred = 'ok'
    return box, pred

In [378]:
def check_contain(img_shape, cnt):
    center_bound = 100
    (x,y), radius = cv2.minEnclosingCircle(cnt)
    center = (int(x), int(y))
    radius = int(radius)
    center_diff = ((int(img_shape[0]/2)-int(y))**2+(int(img_shape[1]/2) -int(x))**2)**0.5
    # print(center, (img_shape[1]/2,img_shape[0]/2), center_diff)
    if center_diff < center_bound:
        return True
    return False

In [379]:
def find_contours(img, ip = None):
    # img = cv2.resize(img, (0,0), fx = 0.3, fy = 0.3, interpolation=cv2.INTER_CUBIC)
    item_img, item_gray, item_bin = preproces(img)
    col = img_mask(item_img)
    # contour, hierachy = cv2.findContours(item_bin, cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    contour, hierachy = cv2.findContours(item_bin, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    result = False
    for i, cnt in enumerate(contour):

        if cv2.contourArea(cnt) > 1500000:
            if check_contain(item_img.shape, cnt):
                cv2.drawContours(item_img, [cnt], 0,(255,0,0),5)
                result = True
                break
        

    if ip:
        cv2.putText(img, ip, (10,60),cv2.FONT_HERSHEY_SIMPLEX,3,(0,255,0),3)
    
    pred = 'NG'
    if result:
        box, pred = cnt_test(cnt)
        # print(box)
        j=1
        cols = [(255,255,255),(0,0,255),(0,255,0),(255,0,0)]
        for i in box[1:3]:
            cv2.circle(item_img,i,5,cols[j],10) 
            j+=1 
        
    cv2.putText(item_img, 'predicted '+pred, (10,60),cv2.FONT_HERSHEY_SIMPLEX,3,(0,255,0),3)


    # print(item_img.shape)
    # cv2.imshow('img', img)
    cv2.imshow('item_img', cv2.resize(item_img, (0,0), fx = 0.3, fy = 0.3, interpolation=cv2.INTER_CUBIC))
    # cv2.imshow('col', col)
    # cv2.imshow('bin', item_bin)
    
    key_val = cv2.waitKey(0)
    cv2.destroyAllWindows()

    return key_val



In [380]:
img_ng = cv2.imread('../team/images/true_ng/GSY827AN7D2341_AAO08637K_PKT11_CM1EQSUA0012_20220714014230_DirectLight_NG.jpg')
img_ng

In [381]:
img_ng = cv2.imread('../team/images/true_ng/GSY827AN7D2341_AAO08637K_PKT15_CM1EQSUA0012_20220714014226_DirectLight_NG.jpg')

In [389]:
find_contours(img_ng, ip = 'NG')
find_contours(img_ng2, ip = 'NG')
find_contours(img_ok, ip = 'ok')
find_contours(img_over, ip = 'ok')

0.998 89.90817991912832
0.982 0.10913468613578126
1.0 0.06837201947086552
0.996 89.86303823813458


13

In [367]:
import os
import random
import time
def check_img(kind = 'overkill'):
    random.seed(time.time_ns()%10000)
    if kind == 'all':
        paths = os.listdir('../team/images/')
        img_paths = []
        for p in paths:
            file_path = '../team/images/'+p+'/'
            img_paths = list(map(lambda x: [file_path+ x, p], os.listdir(file_path)))
        while True:
            ip = random.choice(img_paths)
            img = cv2.imread(ip[0])
            
            key_val= find_contours(img, ip =ip[1])
            
            if key_val == 113:
                break

    else:
        file_path = '../team/images/'+kind+'/'
        img_paths = os.listdir(file_path)
        for i in range(len(img_paths)):
            img = cv2.imread(file_path+img_paths[i])
            key_val = find_contours(img,  ip = kind)
            if key_val == 113: # ord('q')
                break

In [191]:
check_img('true_ng')

[[[   0    0]]

 [[   0  493]]

 [[   1  493]]

 ...

 [[1746    4]]

 [[1746    1]]

 [[1745    0]]]
[[[355 328]]

 [[355 331]]

 [[357 333]]

 ...

 [[364 335]]

 [[364 334]]

 [[358 328]]]


In [390]:
check_img('overkill')

0.999 0.06837201947086552
0.996 89.86303823813458
1.0 0.2065944634734015
1.002 89.58944809699032
0.999 89.77191291062489


C:\Users\jhs85\AppData\Local\Temp\ipykernel_9936\72081498.py:7: RuntimeWarning: divide by zero encountered in longlong_scalars
  radian = math.atan(yd/xd)


0.998 90.0
1.001 89.90810628647117
1.0 0.10255796835506453
1.0 89.81767514362203
1.0 89.90876475941597


In [133]:
check_img('true_ok')

(1231, 1798) (1232.0, 1028.0) 770.0006493503755
(1225, 1003) (1232.0, 1028.0) 25.96150997149434
(1231, 1808) (1232.0, 1028.0) 780.0006410253776
(1219, 1030) (1232.0, 1028.0) 13.152946437965905
(1231, 1793) (1232.0, 1028.0) 765.0006535944921
(1243, 1017) (1232.0, 1028.0) 15.556349186104045
(1231, 1789) (1232.0, 1028.0) 761.0006570299397
(1225, 998) (1232.0, 1028.0) 30.805843601498726


In [27]:
def img_resize(img, resize_size = 1600):
    width_origin = img.shape[1]
    height_origin = img.shape[0]
    hw_list = [height_origin, width_origin]

    img_ratio = float(resize_size)/ max(hw_list)
    print(img_ratio)
    max_index = hw_list.index(max(hw_list))
    if max_index ==0:
        img = cv2.resize(img, (int(width_origin * img_ratio), resize_size),interpolation=cv2.INTER_CUBIC)
    else:
        img = cv2.resize(img, (resize_size,int(height_origin * img_ratio)),interpolation=cv2.INTER_CUBIC)

    return img
